In [43]:
import torch
import torch_pruning as tp
from yolox.models import create_yolox_model

model = create_yolox_model(num_classes=20,name = 'yolox_custom',  
exp_path = '/workspace/tensorrt/YOLOX/exps/default/custom2/type2/all50_stem/yolox_voc_tiny_all50_stem.py',   # 오리진 모델 불러와서 자르기
ckpt_path = '/workspace/tensorrt/YOLOX/fpgm_2/type2/first/yolox_voc_tiny_all50_stem/34.65.pth'
).cuda()
# 1. setup strategy (L1 Norm)
strategy = tp.strategy.L1Strategy() # or tp.strategy.RandomStrategy()

In [44]:

# 2. build dependency graph for resnet18
DG = tp.DependencyGraph()
DG.build_dependency(model, example_inputs=torch.randn(1,3,224,224).cuda())



In [45]:

temp = []
temp2 = list(model.state_dict().keys())
num_list = [0,1,2,3]
#print(temp2)
for i in range(len(temp2)):
        if 'bn' in temp2[i]:
            continue
        if 'weight' in temp2[i]:
            temp2[i] = temp2[i].replace('.weight','')
            temp2[i] = temp2[i].replace('0.0','0[0]')
            temp2[i] = temp2[i].replace('0.1','0[1]')
            temp2[i] = temp2[i].replace('1.0','1[0]')
            temp2[i] = temp2[i].replace('1.1','1[1]')
            temp2[i] = temp2[i].replace('2.0','2[0]')
            temp2[i] = temp2[i].replace('2.1','2[1]')
            temp2[i] = temp2[i].replace('3.0','3[0]')
            temp2[i] = temp2[i].replace('3.1','3[1]')
            temp2[i] = temp2[i].replace('4.0','4[0]')
            temp2[i] = temp2[i].replace('4.1','4[1]')
            temp2[i] = temp2[i].replace('5.0','5[0]')
            temp2[i] = temp2[i].replace('5.1','5[1]')
            temp2[i] = temp2[i].replace('5.2','5[2]')
            temp2[i] = temp2[i].replace('m.0','m[0]')
            temp2[i] = temp2[i].replace('m.1','m[1]')
            temp2[i] = temp2[i].replace('m.2','m[2]')
            temp.append(temp2[i])

print(temp)
# backbone 만 프루닝


['backbone.backbone.stem.conv.conv', 'backbone.backbone.dark2[0].conv', 'backbone.backbone.dark2[1].conv1.conv', 'backbone.backbone.dark2[1].conv2.conv', 'backbone.backbone.dark2[1].conv3.conv', 'backbone.backbone.dark2[1].m[0].conv1.conv', 'backbone.backbone.dark2[1].m[0].conv2.conv', 'backbone.backbone.dark3[0].conv', 'backbone.backbone.dark3[1].conv1.conv', 'backbone.backbone.dark3[1].conv2.conv', 'backbone.backbone.dark3[1].conv3.conv', 'backbone.backbone.dark3[1].m[0].conv1.conv', 'backbone.backbone.dark3[1].m[0].conv2.conv', 'backbone.backbone.dark3[1].m[1].conv1.conv', 'backbone.backbone.dark3[1].m[1].conv2.conv', 'backbone.backbone.dark3[1].m[2].conv1.conv', 'backbone.backbone.dark3[1].m[2].conv2.conv', 'backbone.backbone.dark4[0].conv', 'backbone.backbone.dark4[1].conv1.conv', 'backbone.backbone.dark4[1].conv2.conv', 'backbone.backbone.dark4[1].conv3.conv', 'backbone.backbone.dark4[1].m[0].conv1.conv', 'backbone.backbone.dark4[1].m[0].conv2.conv', 'backbone.backbone.dark4[1].m

In [46]:
print(model)

YOLOX(
  (backbone): YOLOPAFPN(
    (backbone): CSPDarknet(
      (stem): Focus(
        (conv): BaseConv(
          (conv): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
      )
      (dark2): Sequential(
        (0): BaseConv(
          (conv): Conv2d(24, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (1): CSPLayer(
          (conv1): BaseConv(
            (conv): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
            (act): SiLU(inplace=True)
          )
          (conv2): BaseConv(
            (conv): Conv2d(48, 24, kernel_siz

In [47]:
from pruning_imagenet import Mask

In [48]:
m = Mask(model)
m.mask_index = [24,27,30,33,39,45,54,57,60,63,69,75,90,93,99,102,96  # backbone conv3 30 60 96
                            ,105,108,111,114,117,120,123,141,159] # neck

m.mask_index = [24,27,30,33,39,45,54,57,60,63,69,75,90,93,99,102,96
                            ,105,108,111,114,117,120,123,141,159,126,129,132,135,138,144,147,150,153,156,162,165,168,171,174, #neck
                           ]# 231,234,237, #stems
                           # 177,180,183,186,189,192,195,198,201,204,207,210] # head
        

In [49]:
m.init_length()

In [50]:
m.init_mask(1, 1/4)
print(m.mask_index)

filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done
filter codebook done
similar index done


In [51]:
m.do_mask()
m.do_similar_mask()

mask Done
mask similar Done


In [52]:
m.if_zero()
print(m.mask_index)
print(m.similar_matrix)

layer: 24, number of nonzero weight is 1728, zero is 576
layer: 27, number of nonzero weight is 1728, zero is 576
layer: 30, number of nonzero weight is 1728, zero is 576
layer: 33, number of nonzero weight is 432, zero is 144
layer: 39, number of nonzero weight is 432, zero is 144
layer: 45, number of nonzero weight is 432, zero is 144
layer: 54, number of nonzero weight is 6912, zero is 2304
layer: 57, number of nonzero weight is 6912, zero is 2304
layer: 60, number of nonzero weight is 6912, zero is 2304
layer: 63, number of nonzero weight is 1728, zero is 576
layer: 69, number of nonzero weight is 1728, zero is 576
layer: 75, number of nonzero weight is 1728, zero is 576
layer: 90, number of nonzero weight is 27648, zero is 9216
layer: 93, number of nonzero weight is 27648, zero is 9216
layer: 96, number of nonzero weight is 27648, zero is 9216
layer: 99, number of nonzero weight is 6912, zero is 2304
layer: 102, number of nonzero weight is 62208, zero is 20736
layer: 105, number o

In [53]:
model = m.model


In [54]:
dict ={}


for key, value in model.state_dict().items():
    temp = []
    if len(value.shape) ==4:
        print('pruning channel and layer name : ',len(value),',', key)
        for i in range(len(value)):
            if value[i].sum() == 0: 
                print('pruning channex index : ', i)
                #temp = torch.norm(value.view(len(value), -1) , p= 1, dim=1)
                #print(temp)
                #dict[key].append(i)
                temp.append(i)
    if temp:   
        dict[key]  = temp



pruning channel and layer name :  24 , backbone.backbone.stem.conv.conv.weight
pruning channel and layer name :  48 , backbone.backbone.dark2.0.conv.weight
pruning channel and layer name :  24 , backbone.backbone.dark2.1.conv1.conv.weight
pruning channel and layer name :  24 , backbone.backbone.dark2.1.conv2.conv.weight
pruning channel and layer name :  48 , backbone.backbone.dark2.1.conv3.conv.weight
pruning channel and layer name :  24 , backbone.backbone.dark2.1.m.0.conv1.conv.weight
pruning channel and layer name :  24 , backbone.backbone.dark2.1.m.0.conv2.conv.weight
pruning channel and layer name :  96 , backbone.backbone.dark3.0.conv.weight
pruning channel and layer name :  24 , backbone.backbone.dark3.1.conv1.conv.weight
pruning channex index :  8
pruning channex index :  10
pruning channex index :  12
pruning channex index :  17
pruning channex index :  21
pruning channex index :  22
pruning channel and layer name :  24 , backbone.backbone.dark3.1.conv2.conv.weight
pruning cha

In [55]:
print(dict)
#del dict['backbone.backbone.dark3.1.conv3.conv.weight'][0]

{'backbone.backbone.dark3.1.conv1.conv.weight': [8, 10, 12, 17, 21, 22], 'backbone.backbone.dark3.1.conv2.conv.weight': [11, 13, 15, 17, 19, 21], 'backbone.backbone.dark3.1.conv3.conv.weight': [3, 4, 9, 15, 17, 19, 24, 25, 29, 34, 41, 43], 'backbone.backbone.dark3.1.m.0.conv1.conv.weight': [0, 1, 2, 8, 12, 21], 'backbone.backbone.dark3.1.m.1.conv1.conv.weight': [0, 2, 4, 8, 21, 22], 'backbone.backbone.dark3.1.m.2.conv1.conv.weight': [4, 8, 10, 18, 20, 21], 'backbone.backbone.dark4.1.conv1.conv.weight': [2, 5, 15, 20, 25, 29, 30, 32, 35, 37, 44, 47], 'backbone.backbone.dark4.1.conv2.conv.weight': [0, 2, 12, 13, 15, 21, 27, 29, 31, 32, 39, 40], 'backbone.backbone.dark4.1.conv3.conv.weight': [1, 11, 12, 15, 16, 18, 19, 26, 28, 36, 39, 40, 42, 46, 47, 50, 53, 54, 66, 72, 78, 82, 84, 89], 'backbone.backbone.dark4.1.m.0.conv1.conv.weight': [1, 4, 6, 16, 20, 21, 24, 27, 30, 33, 34, 46], 'backbone.backbone.dark4.1.m.1.conv1.conv.weight': [2, 12, 15, 17, 18, 19, 22, 24, 25, 27, 40, 44], 'backbo

In [56]:

temp = []
temp2 = list(dict.keys())
for i in range(len(temp2)):
        if 'bn' in temp2[i]:
            continue
        if 'weight' in temp2[i]:
            temp2[i] = temp2[i].replace('.weight','')
            temp2[i] = temp2[i].replace('.0.0','[0][0]')
            temp2[i] = temp2[i].replace('.0.1','[0][1]')
            temp2[i] = temp2[i].replace('.1.0','[1][0]')
            temp2[i] = temp2[i].replace('.1.1','[1][1]')
            temp2[i] = temp2[i].replace('.2.0','[2][0]')
            temp2[i] = temp2[i].replace('.2.1','[2][1]')
            temp2[i] = temp2[i].replace('3.0','3[0]')
            temp2[i] = temp2[i].replace('3.1','3[1]')
            temp2[i] = temp2[i].replace('4.0','4[0]')
            temp2[i] = temp2[i].replace('4.1','4[1]')
            temp2[i] = temp2[i].replace('5.0','5[0]')
            temp2[i] = temp2[i].replace('5.1','5[1]')
            temp2[i] = temp2[i].replace('5.2','5[2]')
            temp2[i] = temp2[i].replace('m.0','m[0]')
            temp2[i] = temp2[i].replace('m.1','m[1]')
            temp2[i] = temp2[i].replace('m.2','m[2]')
            temp2[i] = temp2[i].replace('.0','[0]')
            temp2[i] = temp2[i].replace('.1','[1]')
            temp2[i] = temp2[i].replace('.2','[2]')
            temp.append(temp2[i])

#temp.insert(27,'head.stems[0].conv')
#temp.insert(28,'head.stems[1].conv')
#temp.insert(29,'head.stems[2].conv')

import copy
key = []
temp2 = copy.deepcopy(temp)
for i in range(len(temp2)):
      
            temp2[i] = temp2[i].replace('[0][0]','.0.0')
            temp2[i] = temp2[i].replace('[0][1]','.0.1',)
            temp2[i] = temp2[i].replace('[1][0]','.1.0',)
            temp2[i] = temp2[i].replace('[1][1]','.1.1',)
            temp2[i] = temp2[i].replace('[2][0]','.2.0',)
            temp2[i] = temp2[i].replace('[2][1]','.2.1',)
            temp2[i] = temp2[i].replace('3[0]','3.0',)
            temp2[i] = temp2[i].replace('3[1]','3.1')
            temp2[i] = temp2[i].replace('4[0]','4.0',)
            temp2[i] = temp2[i].replace('4[1]','4.1',)
            temp2[i] = temp2[i].replace('5[0]','5.0',)
            temp2[i] = temp2[i].replace('5[1]','5.1',)
            temp2[i] = temp2[i].replace('5[2]','5.2',)
            temp2[i] = temp2[i].replace('m[0]','m.0',)
            temp2[i] = temp2[i].replace('m[1]','m.1',)
            temp2[i] = temp2[i].replace('m[2]','m.2',)
            temp2[i] = temp2[i].replace('[0]','.0',)
            temp2[i] = temp2[i].replace('[1]','.1',)
            temp2[i] = temp2[i].replace('[2]','.2',)
            key.append(temp2[i])

print(key)
print(temp)

['backbone.backbone.dark3.1.conv1.conv', 'backbone.backbone.dark3.1.conv2.conv', 'backbone.backbone.dark3.1.conv3.conv', 'backbone.backbone.dark3.1.m.0.conv1.conv', 'backbone.backbone.dark3.1.m.1.conv1.conv', 'backbone.backbone.dark3.1.m.2.conv1.conv', 'backbone.backbone.dark4.1.conv1.conv', 'backbone.backbone.dark4.1.conv2.conv', 'backbone.backbone.dark4.1.conv3.conv', 'backbone.backbone.dark4.1.m.0.conv1.conv', 'backbone.backbone.dark4.1.m.1.conv1.conv', 'backbone.backbone.dark4.1.m.2.conv1.conv', 'backbone.backbone.dark5.2.conv1.conv', 'backbone.backbone.dark5.2.conv2.conv', 'backbone.backbone.dark5.2.conv3.conv', 'backbone.backbone.dark5.2.m.0.conv1.conv', 'backbone.backbone.dark5.2.m.0.conv2.conv', 'backbone.lateral_conv0.conv', 'backbone.C3_p4.conv1.conv', 'backbone.C3_p4.conv2.conv', 'backbone.C3_p4.conv3.conv', 'backbone.C3_p4.m.0.conv1.conv', 'backbone.C3_p4.m.0.conv2.conv', 'backbone.reduce_conv1.conv', 'backbone.C3_p3.conv1.conv', 'backbone.C3_p3.conv2.conv', 'backbone.C3_p3

In [57]:
value = list(dict.values())
value

[[8, 10, 12, 17, 21, 22],
 [11, 13, 15, 17, 19, 21],
 [3, 4, 9, 15, 17, 19, 24, 25, 29, 34, 41, 43],
 [0, 1, 2, 8, 12, 21],
 [0, 2, 4, 8, 21, 22],
 [4, 8, 10, 18, 20, 21],
 [2, 5, 15, 20, 25, 29, 30, 32, 35, 37, 44, 47],
 [0, 2, 12, 13, 15, 21, 27, 29, 31, 32, 39, 40],
 [1,
  11,
  12,
  15,
  16,
  18,
  19,
  26,
  28,
  36,
  39,
  40,
  42,
  46,
  47,
  50,
  53,
  54,
  66,
  72,
  78,
  82,
  84,
  89],
 [1, 4, 6, 16, 20, 21, 24, 27, 30, 33, 34, 46],
 [2, 12, 15, 17, 18, 19, 22, 24, 25, 27, 40, 44],
 [5, 6, 7, 9, 12, 13, 15, 25, 35, 37, 41, 44],
 [0,
  1,
  4,
  6,
  9,
  15,
  16,
  17,
  18,
  25,
  29,
  38,
  39,
  40,
  43,
  56,
  65,
  69,
  70,
  71,
  82,
  84,
  85,
  87],
 [0,
  1,
  2,
  8,
  14,
  15,
  31,
  33,
  34,
  35,
  41,
  42,
  44,
  51,
  54,
  59,
  62,
  63,
  67,
  68,
  73,
  78,
  83,
  89],
 [8,
  10,
  11,
  14,
  38,
  39,
  46,
  52,
  53,
  57,
  60,
  68,
  69,
  70,
  74,
  75,
  80,
  81,
  93,
  94,
  102,
  104,
  105,
  109,
  112,
  113,

In [58]:
for i in range(len(temp)):
    pruning_plan = DG.get_pruning_plan(eval(f"model.{temp[i]}"), tp.prune_conv_out_channel, dict[(key[i]) + '.weight'])
    print(pruning_plan)
    if DG.check_pruning_plan(pruning_plan):
        pruning_plan.exec()     

Conv2d(96, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)

--------------------------------
          Pruning Plan
--------------------------------
User pruning:
[ [DEP] ConvOutChannelPruner on backbone.backbone.dark3.1.conv1.conv (Conv2d(96, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)) => ConvOutChannelPruner on backbone.backbone.dark3.1.conv1.conv (Conv2d(96, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)), Index=[8, 10, 12, 17, 21, 22], metric={'#params': 576}]

Coupled pruning:
[ [DEP] ConvOutChannelPruner on backbone.backbone.dark3.1.conv1.conv (Conv2d(96, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)) => BatchnormPruner on backbone.backbone.dark3.1.conv1.bn (BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)), Index=[8, 10, 12, 17, 21, 22], metric={'#params': 12}]
[ [DEP] BatchnormPruner on backbone.backbone.dark3.1.conv1.bn (BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)) => ElementWiseOpPruner o

In [59]:
print(model)

YOLOX(
  (backbone): YOLOPAFPN(
    (backbone): CSPDarknet(
      (stem): Focus(
        (conv): BaseConv(
          (conv): Conv2d(12, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
      )
      (dark2): Sequential(
        (0): BaseConv(
          (conv): Conv2d(24, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (1): CSPLayer(
          (conv1): BaseConv(
            (conv): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNorm2d(24, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
            (act): SiLU(inplace=True)
          )
          (conv2): BaseConv(
            (conv): Conv2d(48, 24, kernel_siz

In [60]:
ckpt_state = {
                "model": model.state_dict(),
            }

torch.save(ckpt_state, 'type2_0.625_all_v1.pth')


